In [1]:
from tabulate import tabulate

arr = []

class Tree:
    def __init__(self, root, left=None, right=None):
        self.root = root
        self.left = left
        self.right = right
        self.label = None

    def __str__(self):
        return f"{self.root=} | {self.left=} | {self.right=} | {self.label=}"

def get_node(value, left=None, right=None):
    root = Tree(value, left, right)
    return root

def create_label(root, label=None):
    if root.left is None and root.right is None:
        root.label = label
        return label

    left = create_label(root.left, 1)
    right = create_label(root.right, 0)

    if left == right:
        root.label = left + 1
    else:
        root.label = max(left if left is not None else -1, right if right is not None else -1)

    return root.label

def return_opname(value):
    match value:
        case "+":
            return "ADD"
        case "-":
            return "SUB"
        case "*":
            return "MUL"
        case "/":
            return "DIV"
        case _:
            return "Invalid"

def gen_code(root, rstack):
    if root is None:
        return []

    result = []

    if root.left is None and root.right is None:
        if root.label == 1:
            result.append(("MOV", root.root, rstack[-1]))
        return result

    if root.right and root.right.label == 0:
        name = root.right.root
        result += gen_code(root.left, rstack)
        result.append((return_opname(root.root), name, rstack[-1]))
    elif root.right.label > root.left.label and root.left.label < root.label:
        rstack[0], rstack[1] = rstack[1], rstack[0]
        result += gen_code(root.right, rstack)
        r = rstack[-1]
        rstack.pop()
        result += gen_code(root.left, rstack)
        rstack.append(r)
        rstack[0], rstack[1] = rstack[1], rstack[0]
        result.append((return_opname(root.root), r, rstack[-1]))
    elif root.right.label <= root.left.label and root.right.label < root.label:
        result += gen_code(root.left, rstack)
        r = rstack.pop()
        result += gen_code(root.right, rstack)
        result.append((return_opname(root.root), rstack[-1], r))
        rstack.append(r)

    return result

root = get_node("-")
t1 = get_node("+")
t3 = get_node("-")
a = get_node("a")
b = get_node("b")
t2 = get_node("+")
e = get_node("e")
c = get_node("c")
d = get_node("d")
t1.left = a
t1.right = b
root.left = t1
t3.left = e
t2.left = c
t2.right = d
t3.right = t2
root.right = t3

create_label(root)
print("\nThe number of register required:", root.label, "\n")
rstack = []
for i in range(root.label):
    rstack.append("R{i}".format(i=i))
print("Stack: ", rstack)
generated_code = gen_code(root, rstack)

# Print three-address code in table format
print(tabulate(generated_code, headers=["Operator", "Operand 1", "Operand 2/Result"]))



The number of register required: 2 

Stack:  ['R0', 'R1']
Operator    Operand 1    Operand 2/Result
----------  -----------  ------------------
MOV         e            R0
MOV         c            R1
ADD         d            R1
SUB         R1           R0
MOV         a            R1
ADD         b            R1
SUB         R0           R1
